In [4]:
pip install pyspark snowflake-connector-python snowflake-sqlalchemy


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
from pyspark.sql import SparkSession

# Set environment variables
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-17.0.2'
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = r'C:\Program Files\Java\jdk-17.0.2\bin;C:\hadoop\bin;' + os.environ['PATH']

# Path to the MySQL Connector/J JAR file
mysql_connector_jar = r"C:\Users\HP\Downloads\mysql-connector-j-8.4.0\mysql-connector-j-8.4.0.jar"

# Initialize Spark session with the MySQL Connector/J
spark_mariadb = SparkSession.builder \
    .appName("MariaDBIntegration") \
    .config("spark.jars", mysql_connector_jar) \
    .getOrCreate()

# JDBC URL for MariaDB
server = "localhost"
port = 3306
database = "foodpricesdb"
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}"

# Database credentials
user = "root"
password = "root"
jdbc_driver = "com.mysql.cj.jdbc.Driver"

# Create a data frame by reading data from MariaDB via JDBC
sql = "SELECT * FROM foodprices"
food_prices_df = spark_mariadb.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", sql) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", jdbc_driver) \
    .load()

# Check food prices data
print('Food Prices shape (rows, cols):', (food_prices_df.count(), len(food_prices_df.columns)))
food_prices_df.show(5)

spark_mariadb.stop()





Food Prices shape (rows, cols): (2400, 11)
+---+---------+----+-----+--------+-----------------+------------+--------+----------+------------+-------+
| id|  Country|Year|Month|FoodItem|UnitOfMeasurement|AveragePrice|Currency|PriceInUSD|Availability|Quality|
+---+---------+----+-----+--------+-----------------+------------+--------+----------+------------+-------+
|  1|Australia|2018|    1|   Bread|             Loaf|        2.20|     AUD|      1.47|           1| High\r|
|  2|Australia|2018|    2|   Bread|             Loaf|        2.20|     AUD|      1.48|           1| High\r|
|  3|Australia|2018|    3|   Bread|             Loaf|        2.20|     AUD|      1.48|           1| High\r|
|  4|Australia|2018|    4|   Bread|             Loaf|        2.21|     AUD|      1.48|           1| High\r|
|  5|Australia|2018|    5|   Bread|             Loaf|        2.21|     AUD|      1.48|           1| High\r|
+---+---------+----+-----+--------+-----------------+------------+--------+----------+-------

In [6]:
# Perform any necessary transformations here
# For example, let's assume we want to select certain columns and add a source column
# transformed_df = food_prices_df.select("Country", "Year", "Month", "Food Item", "Quality").withColumn("source", lit("mariadb"))


In [7]:

# JDBC URL for Azure SQL Database
# jdbc_url_azure = "jdbc:sqlserver://your_server.database.windows.net:1433;database=your_database"

# Load the transformed data into Azure SQL Database
# transformed_df.write.format("jdbc") \
#    .option("url", jdbc_url_azure) \
#    .option("dbtable", "your_final_table") \
#    .option("user", "your_username") \
#    .option("password", "your_password") \
#    .save()

# Stop the Spark session
# spark.stop()

In [8]:
from pyspark.sql import SparkSession

# Update these paths to the actual locations of the JAR files
jdbc_path = r"C:\\Users\\HP\\Downloads\\snowflake jar files\\snowflake-jdbc-3.13.9.jar"
spark_snowflake_path = r"C:\\Users\\HP\\Downloads\\snowflake jar files\\spark-snowflake_2.12-2.9.0-spark_3.1.jar"

# Create a Spark session
spark_snowflakes = SparkSession.builder \
    .appName("SnowflakePySpark") \
    .config("spark.jars", f"{jdbc_path},{spark_snowflake_path}") \
    .config("spark.executor.extraClassPath", f"{jdbc_path};{spark_snowflake_path}") \
    .config("spark.driver.extraClassPath", f"{jdbc_path};{spark_snowflake_path}") \
    .getOrCreate()
    
# Snowflake connection options
snowflake_options = {
    "sfURL": "https://dvpekou-si12121.snowflakecomputing.com",
    "sfUser": "SHIVANIAFK",
    "sfPassword": "Shivani@09",
    "sfDatabase": "Snowflake_database",
    "sfSchema": "Snowflake_schema",
    "sfWarehouse": "my_warehouse"
}

# Read data from the modified_HP table
df_modified_HP = spark_snowflakes.read \
    .format("net.snowflake.spark.snowflake") \
    .options(**snowflake_options) \
    .option("dbtable", "modified_HP") \
    .load()
    

spark_snowflakes.stop()

Py4JJavaError: An error occurred while calling o131.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: net.snowflake.spark.snowflake. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:172)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMe

In [1]:
pip install pyspark snowflake-connector-python snowflake-sqlalchemy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

# Cell 1: Installs

# Cell 2: Spark Session Configuration
import os
from pyspark.sql import SparkSession

# Set environment variables
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-17.0.2'
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = r'C:\Program Files\Java\jdk-17.0.2\bin;C:\hadoop\bin;' + os.environ['PATH']

# Paths to the JAR files
mysql_connector_jar = r"C:\Users\HP\Downloads\mysql-connector-j-8.4.0\mysql-connector-j-8.4.0.jar"
snowflake_jdbc_path = r"C:\\Users\\HP\\Downloads\\snowflake jar files\\snowflake-jdbc-3.13.9.jar"
spark_snowflake_path = r"C:\\Users\\HP\\Downloads\\snowflake jar files\\spark-snowflake_2.12-2.9.0-spark_3.1.jar"

# Initialize Spark session with both MariaDB and Snowflake JARs
spark = SparkSession.builder \
    .appName("DataIntegration") \
    .config("spark.jars", f"{mysql_connector_jar},{snowflake_jdbc_path},{spark_snowflake_path}") \
    .config("spark.executor.extraClassPath", f"{mysql_connector_jar};{snowflake_jdbc_path};{spark_snowflake_path}") \
    .config("spark.driver.extraClassPath", f"{mysql_connector_jar};{snowflake_jdbc_path};{spark_snowflake_path}") \
    .getOrCreate()

# Cell 3: MariaDB
# JDBC URL for MariaDB
server = "localhost"
port = 3306
database = "foodpricesdb"
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}"

# Database credentials
user = "root"
password = "root"
jdbc_driver = "com.mysql.cj.jdbc.Driver"

# Connector with MariaDB via JDBC
sql = "SELECT * FROM foodprices"
food_prices_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", sql) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", jdbc_driver) \
    .load()
    
transformed_df = food_prices_df.select("Country", "Year", "Month", "Food Item", "Quality") \
    .withColumn("source", lit("mariadb"))

# Check food prices data
print('Food Prices shape (rows, cols):', (food_prices_df.count(), len(food_prices_df.columns)))
food_prices_df.show(5)

# Connector with Snowflake
snowflake_options = {
    "sfURL": "https://dvpekou-si12121.snowflakecomputing.com",
    "sfUser": "SHIVANIAFK",
    "sfPassword": "Shivani@09",
    "sfDatabase": "Snowflake_database",
    "sfSchema": "Snowflake_schema",
    "sfWarehouse": "my_warehouse"
}

# Read data from the modified_HP table
df_modified_HP = spark.read \
    .format("net.snowflake.spark.snowflake") \
    .options(**snowflake_options) \
    .option("dbtable", "modified_HP") \
    .load()

# Check Snowflake data
df_modified_HP.show(5)

# Use a simple query to fetch data from Snowflake
df_modified_file_inflation_rate = spark.read \
    .format("net.snowflake.spark.snowflake") \
    .options(**snowflake_options) \
    .option("query", "SELECT * FROM INFLATION_RATE") \
    .load()
    
df_modified_file_inflation_rate.show()

# Stop Spark session
spark.stop()


# Here WE are combining all the data from Datasources 1, 2, 3, etc

# WE end up with the following tables / rows for the tables

# Here we connect to Azure SQL

# Here we are pushing the final rows to Azure SQL



Food Prices shape (rows, cols): (2400, 11)
+---+---------+----+-----+--------+-----------------+------------+--------+----------+------------+-------+
| id|  Country|Year|Month|FoodItem|UnitOfMeasurement|AveragePrice|Currency|PriceInUSD|Availability|Quality|
+---+---------+----+-----+--------+-----------------+------------+--------+----------+------------+-------+
|  1|Australia|2018|    1|   Bread|             Loaf|        2.20|     AUD|      1.47|           1| High\r|
|  2|Australia|2018|    2|   Bread|             Loaf|        2.20|     AUD|      1.48|           1| High\r|
|  3|Australia|2018|    3|   Bread|             Loaf|        2.20|     AUD|      1.48|           1| High\r|
|  4|Australia|2018|    4|   Bread|             Loaf|        2.21|     AUD|      1.48|           1| High\r|
|  5|Australia|2018|    5|   Bread|             Loaf|        2.21|     AUD|      1.48|           1| High\r|
+---+---------+----+-----+--------+-----------------+------------+--------+----------+-------